In [ ]:
import os

import numpy as np
import pandas as pd
import scipy.stats as sts

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

import lib.exp4 as exp4
from lib.reconstruction.config import colors, set_library_options
import lib.reconstruction.errors as errs
import lib.reconstruction.features as ftrs
import lib.reconstruction.neighbors as nbrs
from lib.reconstruction.permutation_tests import PermutationTestBetween, PermutationTestWithin

import lib.utility_functions as uf
import lib.util.datatype_utilities as dtutils

# Config
set_library_options()

% matplotlib inline

In [ ]:
sns.palplot(colors.values())

In [ ]:
tidy = pd.read_csv('./tidy_data.csv', index_col=0)
tidy['Condition'] = tidy['Condition'].map(lambda x: 'Untrained' if x == 'Naive' else x)
tidy['Position ID'] = tidy['Position ID'].map(int)

vals = ['Black Position', 'White Position', 'Is Real', 'Num Pieces']

board_set = tidy.pivot_table(index='Position ID', values=vals, aggfunc=lambda x: x.unique()[0])[vals]

In [ ]:
tidy['Total Error Rate'] = tidy['Total Errors'] / 36
tidy['Type I Rate'] = tidy['Type I Errors'] / (36 - tidy['Num Pieces'])
tidy['Type II Rate'] = tidy['Type II Errors'] / tidy['Num Pieces']
tidy['Type III Rate'] = tidy['Type III Errors'] / tidy['Num Pieces']

In [ ]:
between_permutation_test = PermutationTestBetween(tidy, num_resamples=1000)
    
samples, axes = between_permutation_test('Type III Rate')

In [ ]:
within_permutation_test = PermutationTestWithin(tidy, num_resamples=1000)

samples, axes = within_permutation_test('Type III Rate')

In [ ]:
tidy.head()